## BRONZE TO SILVER LAYER


In [1]:
import pandas as pd
from common_utilities import global_path
import datetime

### GOLD LAYER - PROCESS HOLDING RECORDS HISTORY


In [2]:
# Load trade history from CSV into DataFrame
df = pd.read_csv(global_path.tradehistory_gold_file_path)

# Filter for specific segments
df = df[df["segment"].isin(["EQ", "MF"])]

df = df[
    [
        "datetime",
        "segment",
        "stock_name",
        "avg_price",
        "holding_quantity",
        "holding_amount",
    ]
]

In [3]:
# Convert 'datetime' column to datetime objects
df["datetime"] = pd.to_datetime(df["datetime"])

# Add a 'date' column by extracting the date part from 'datetime'
df["date"] = df["datetime"].dt.date

# Sort DataFrame by 'segment', 'stock_name', and 'datetime'
df = df.sort_values(by=["segment", "stock_name", "datetime"])

# Get maximum 'datetime' for each 'date' and 'stock_name' combination
max_datetime_df = (
    df.groupby(["date", "stock_name"])["datetime"].max().reset_index()
)

# Retain only rows with maximum datetime for each 'date' and 'stock_name'
df = df.merge(max_datetime_df, on=["date", "stock_name", "datetime"])

In [4]:
# Sort the DataFrame by 'segment', 'stock_name', and 'date'
df = df.sort_values(by=["segment", "stock_name", "date"]).reset_index(drop=True)

# Process each stock name separately
stock_names = df["stock_name"].unique()
result = []

for stock_name in stock_names:
    stock_data = df[df["stock_name"] == stock_name].copy()

    # Set 'date' as index and reindex to include all dates up to today
    stock_data = stock_data.set_index("date")
    date_range = pd.date_range(
        start=stock_data.index.min(),
        end=datetime.datetime.today().date(),
        freq="D",
    )
    stock_data = stock_data.reindex(date_range, method="ffill")

    # Reset index to bring 'date' back as a column
    stock_data = stock_data.reset_index().rename(columns={"index": "date"})
    result.append(stock_data)

# Combine processed DataFrames into one
df = pd.concat(result, ignore_index=True)

In [5]:
# Load stock prices and merge with the main DataFrame
df_StockPrice = pd.read_csv(global_path.stockprice_silver_file_path)
df_StockPrice = df_StockPrice[["date", "stock_name", "close"]]
df_StockPrice["date"] = pd.to_datetime(df_StockPrice["date"])

# Merge stock price data
df = pd.merge(df, df_StockPrice, on=["date", "stock_name"], how="left")

# Rename columns for clarity and calculate current value
df = df.rename(
    columns={
        "holding_amount": "investment",
        "holding_quantity": "quantity",
        "close": "current_price",
    }
)
df["current_value"] = df["current_price"] * df["quantity"]

# Filter out rows with zero 'holding_quantity'
df = df[(df["investment"] != 0) & (df["current_value"]!= 0)]

In [6]:
# Final sorting and column selection
df = df.sort_values(by=["segment", "stock_name", "date"]).reset_index(drop=True)
df = df[
    [
        "date",
        "segment",
        "stock_name",
        "quantity",
        "avg_price",
        "investment",
        "current_price",
        "current_value",
    ]
]

# Save the final DataFrame to a CSV file
df.to_csv(global_path.holdings_gold_file_path, index=None)

# Display DataFrame information and print success message
df.info()
print("GOLD Layer CSV file for Holdings successfully created at:")
print(global_path.holdings_gold_file_path.resolve())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5863 entries, 0 to 5862
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           5863 non-null   datetime64[ns]
 1   segment        5863 non-null   object        
 2   stock_name     5863 non-null   object        
 3   quantity       5863 non-null   int64         
 4   avg_price      5863 non-null   float64       
 5   investment     5863 non-null   float64       
 6   current_price  3759 non-null   float64       
 7   current_value  3759 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 366.6+ KB
GOLD Layer CSV file for Holdings successfully created at:
C:\Users\prashant.tripathi\Code\Upstox\DATA\GOLD\Holdings\Holdings_data.csv
